# Pipestat CLI

**Before following this tutorial please make sure you're familiar with more information-rich "Pipestat Python API" tutorial.**

## Usage reference

To learn about the usage `pipestat` usage use `--help`/`-h` option on any level:

In [1]:
pipestat -h

version: 0.0.1
usage: pipestat [-h] [--version] [--silent] [--verbosity V] [--logdev]
                {report,inspect,remove,retrieve} ...

pipestat - report pipeline results

positional arguments:
  {report,inspect,remove,retrieve}
    report              Report a result.
    inspect             Inspect a database.
    remove              Remove a result.
    retrieve            Retrieve a result.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  --silent              Silence logging. Overrides verbosity.
  --verbosity V         Set logging level (1-5 or logging module level name)
  --logdev              Expand content of logging message format.

Pipestat standardizes reporting of pipeline results and pipeline status
management. It formalizes a way for pipeline developers and downstream tools
developers to communicate -- results produced by a pipeline can easily and
reliably become an input for

In [2]:
pipestat report -h

usage: pipestat report [-h] -n N (-f F | -c C | -a) -s S [--status-schema ST]
                       [--flag-dir FD] -i I -r R -v V [-o] [-t]

Report a result.

optional arguments:
  -h, --help                   show this help message and exit
  -n N, --namespace N          Name of the pipeline to report result for
  -f F, --results-file F       Path to the YAML file where the results will be stored.
                               This file will be used as pipestat backend and to restore
                               the reported results across sesssions
  -c C, --database-config C    Path to the YAML file with PostgreSQL database
                               configuration. Please refer to the documentation for the
                               file format requirements.
  -a, --database-only          Whether the reported data should not be stored in the
                               memory, only in the database.
  -s S, --schema S             Path to the schema that defines the re

In [3]:
pipestat retrieve -h

usage: pipestat retrieve [-h] -n N (-f F | -c C | -a) [-s S] [--status-schema ST]
                         [--flag-dir FD] -i I -r R

Retrieve a result.

optional arguments:
  -h, --help                   show this help message and exit
  -n N, --namespace N          Name of the pipeline to report result for
  -f F, --results-file F       Path to the YAML file where the results will be stored.
                               This file will be used as pipestat backend and to restore
                               the reported results across sesssions
  -c C, --database-config C    Path to the YAML file with PostgreSQL database
                               configuration. Please refer to the documentation for the
                               file format requirements.
  -a, --database-only          Whether the reported data should not be stored in the
                               memory, only in the database.
  -s S, --schema S             Path to the schema that defines the results t

In [4]:
pipestat remove -h

usage: pipestat remove [-h] -n N (-f F | -c C | -a) [-s S] [--status-schema ST]
                       [--flag-dir FD] -i I -r R

Remove a result.

optional arguments:
  -h, --help                   show this help message and exit
  -n N, --namespace N          Name of the pipeline to report result for
  -f F, --results-file F       Path to the YAML file where the results will be stored.
                               This file will be used as pipestat backend and to restore
                               the reported results across sesssions
  -c C, --database-config C    Path to the YAML file with PostgreSQL database
                               configuration. Please refer to the documentation for the
                               file format requirements.
  -a, --database-only          Whether the reported data should not be stored in the
                               memory, only in the database.
  -s S, --schema S             Path to the schema that defines the results that ca

In [5]:
pipestat inspect -h

usage: pipestat inspect [-h] -n N (-f F | -c C | -a) [-s S] [--status-schema ST]
                        [--flag-dir FD] [-d]

Inspect a database.

optional arguments:
  -h, --help                 show this help message and exit
  -n N, --namespace N        Name of the pipeline to report result for
  -f F, --results-file F     Path to the YAML file where the results will be stored. This
                             file will be used as pipestat backend and to restore the
                             reported results across sesssions
  -c C, --database-config C  Path to the YAML file with PostgreSQL database configuration.
                             Please refer to the documentation for the file format
                             requirements.
  -a, --database-only        Whether the reported data should not be stored in the memory,
                             only in the database.
  -s S, --schema S           Path to the schema that defines the results that can be
                 

## Usage demonstration

### Reporting

Naturally, the command line interface provides access to all the Python API functionalities of `pipestat`. So, for example, to report a result and back the object by a file use:

In [6]:
temp_file=`mktemp`
pipestat report -f $temp_file -n test -r sample1 -i number_of_things -v 100 -s ../tests/data/sample_output_schema.yaml --try-convert

Reported records for 'sample1' in 'test' namespace:
 - number_of_things: 100


The result has been reported and the database file has been updated:

In [7]:
cat $temp_file

test:
  sample1:
    number_of_things: 100


Let's report another result:

In [8]:
pipestat report -f $temp_file -n test -r sample1 -i percentage_of_things -v 1.1 -s ../tests/data/sample_output_schema.yaml --try-convert

Reported records for 'sample1' in 'test' namespace:
 - percentage_of_things: 1.1


In [9]:
cat $temp_file

test:
  sample1:
    number_of_things: 100
    percentage_of_things: 1.1


### Inspection

`pipestat inspect` command is a way to briefly look at the general `PipestatManager` state, like number of records, type of backend etc.

In [10]:
pipestat inspect -f $temp_file -n test -s ../tests/data/sample_output_schema.yaml



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.anMBzQ78)
Schema source: /Users/mstolarczyk/code/pipestat/docs_jupyter/../tests/data/sample_output_schema.yaml
Records count: 1


In order to display the contents of the results file or database table associated with the indicated namespace, add `--data` flag:

In [11]:
pipestat inspect --data -f $temp_file -n test -s ../tests/data/sample_output_schema.yaml



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.anMBzQ78)
Schema source: /Users/mstolarczyk/code/pipestat/docs_jupyter/../tests/data/sample_output_schema.yaml
Records count: 1

Data:
test:
  sample1:
    number_of_things: 100
    percentage_of_things: 1.1


### Retrieval

Naturally, the reported results can be retrieved. Just call `pipestat retrieve` to do so:

In [12]:
pipestat retrieve -f $temp_file -n test -r sample1 -i percentage_of_things -s ../tests/data/sample_output_schema.yaml

1.1


### Removal

In order to remove a result call `pipestat remove`:

In [13]:
pipestat remove -f $temp_file -n test -r sample1 -i percentage_of_things -s ../tests/data/sample_output_schema.yaml

Removed result 'percentage_of_things' for record 'sample1' from 'test' namespace


The results file and the state of the `PipestatManager` object reflect the removal:

In [14]:
cat $temp_file

test:
  sample1:
    number_of_things: 100


In [15]:
pipestat inspect --data -f $temp_file -n test -s ../tests/data/sample_output_schema.yaml



PipestatManager (test)
Backend: file (/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.anMBzQ78)
Schema source: /Users/mstolarczyk/code/pipestat/docs_jupyter/../tests/data/sample_output_schema.yaml
Records count: 1

Data:
test:
  sample1:
    number_of_things: 100
